In [3]:
import os, sys, argparse, importlib, time, inspect
import numpy as np
import matplotlib.pyplot as plt
import os.path as osp
if hasattr(__builtins__,'__IPYTHON__'):
    print('Notebook')
    notebook=1
    from tqdm.notebook import tqdm
else:
    print('Not notebook')
    notebook=0
    from tqdm import tqdm
from tensorflow.keras import backend as K
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow as tf
import tensorflow_probability as tfp

gpu_devices = tf.config.list_physical_devices('GPU') 
if len(gpu_devices) > 0:
    print("GPU detected")
    tf.config.experimental.set_memory_growth(gpu_devices[0], True)
else:
    print('No GPU detected')

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.models import load_model, Model
import spektral
from sklearn.preprocessing import normalize
from spektral.data import DisjointLoader, BatchLoader, SingleLoader
from importlib import reload
if notebook:
    import winsound
import dill
import datetime as dt
import plotly

Notebook
GPU detected


In [166]:
import os
import numpy as np
from spektral.layers.convolutional.gcn_conv import GCNConv


os.environ["TF_CPP_MIN_LOG_LEVEL"] = '2'
import tensorflow as tf

from spektral.layers import ECCConv
from spektral.layers.pooling.global_pool import GlobalMaxPool, GlobalAvgPool, GlobalSumPool

from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dense, LeakyReLU, BatchNormalization, Dropout, multiply
from tensorflow.keras.activations import tanh, sigmoid
from tensorflow.sparse import SparseTensor

class glob(Model):
    def __init__(self, n_out = 4, hidden_states=64, conv_layers=2, conv_activation='relu', decode_layers=2, decode_activation='relu', regularization=None, dropout=0.2, batch_norm=True, forward=True, edgeconv=True):
        super().__init__()

    def call(self, inputs, training = False):
        global i_n
        x, a, i = inputs
        glob_avg=tf.math.segment_mean(x,i)
        glob_var=abs(tf.math.subtract(tf.math.segment_mean(multiply([x,x]),i),multiply([glob_avg, glob_avg])))
        glob_max=tf.math.segment_max(x,i)
        glob_min=tf.math.segment_min(x,i)
        tf.print(glob_avg, glob_var, glob_max, glob_min)
        return tf.reduce_mean(glob_avg, axis=1)[:4]
model=glob()

In [167]:
learning_rate = 2.5e-4
warm_up       = 4
batch_size    = 512
epochs        = 20
n_data       = 1000
scenario    = "stop_muon"
patience = 20

################################################
# Setup Model hyperparameters                  # 
################################################

# make in model

################################################
# Setup Other Hyperparameters                   # 
################################################


loss_method   = "von Mises"
n_neighbors   = 6 # SKRIV SELV IND
wandblog      = 0
opt = Adam(learning_rate)

import data_load as dl
graph_data=dl.graph_data
dataset=graph_data(n_data=n_data, restart=0, transform=True, angle=True, unitvec=False)
idx_lists = dataset.index_lists
# Split data
dataset_train = dataset[idx_lists[0]]
dataset_val   = dataset[idx_lists[1]]
dataset_test  = dataset[idx_lists[2]]

loader_train = DisjointLoader(dataset_train, epochs=epochs, batch_size=batch_size) # the different loaders work very very differently, beware
loader_test = DisjointLoader(dataset_test, batch_size=batch_size, epochs=1)
if notebook:
    winsound.Beep(400,300)

Loading data to memory


In [171]:
def loss_func(y_reco, y_true, re=False):
    loss_energy = tf.reduce_mean((tf.subtract(y_reco[:,0], y_true[:,0]))**2)
    kappa=tf.math.abs(y_reco[:,3])
#     tf.print(tf.reduce_mean(kappa))
    zep = y_reco[:,1] #Zenith prediction
    zet = y_true[:,1] #Zenith target
    azp = y_reco[:,2] #Azimuth prediction
    azt = y_true[:,2] #Azimuth target
    s1 = tf.math.sin(zet + azt - azp )
    s2 = tf.math.sin( zet - azt + azp )
    c1 = tf.math.cos( zet - zep )
    c2 = tf.math.cos( zet + zep )
    cos_angle = 0.5*tf.math.abs(tf.math.sin(zep))*( s1 + s2 ) + 0.5*(c1 + c2)
    nlogC = - tf.math.log(kappa) + kappa + tf.math.log( 1 - tf.math.exp( - 2 * kappa) )

    loss_angle = tf.reduce_mean( - kappa*cos_angle + nlogC )
    if not re:
        return loss_angle+loss_energy
    if re:
        return float(loss_angle+loss_energy), [float(loss_energy), float(loss_angle)]

In [172]:
# ################################################
# # Setup functions                            # 
# ################################################

def lr_schedule(epochs = epochs, initial = learning_rate, warm_up=warm_up, decay = 0.9):
    n = 1
    lr = initial
    yield lr
    while n < warm_up:
        lr *= 2
        n  += 1
        yield lr
    while True:
        lr *= decay
        n  += 1 
        yield lr


#make functions into tf functions

@tf.function(input_signature = loader_train.tf_signature(), experimental_relax_shapes = True)
def train_step(inputs, targets):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training = True)
        targets     = tf.cast(targets, tf.float32)
        loss        = loss_func(predictions, targets)
        loss       += sum(model.losses)

    gradients = tape.gradient(loss, model.trainable_variables)
    opt.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

@tf.function(input_signature = loader_test.tf_signature(), experimental_relax_shapes = True)
def test_step(inputs, targets):
    predictions = model(inputs, training = False)
    targets     = tf.cast(targets, tf.float32) 
    out         = loss_func(predictions, targets)

    return predictions, targets, out


def validation(loader):
    loss = 0
    prediction_list, target_list = [], []
    for batch in loader:
        inputs, targets = batch
        predictions, targets, out = test_step(inputs, targets)
        loss           += out
        
        prediction_list.append(predictions)
        target_list.append(targets)
    
    y_reco  = tf.concat(prediction_list, axis = 0)
    y_true  = tf.concat(target_list, axis = 0)
    y_true  = tf.cast(y_true, tf.float32)

    loss = loss_func(y_reco, y_true, re=True)
    
    energy, angle, old= metrics(y_reco, y_true)
    
    return loss, loss_from, [energy, angle, old]

In [173]:
for batch in loader_train:
    inputs, targets = batch
    out             = train_step(inputs, targets)
    loss           += out

ValueError: in user code:

    <ipython-input-169-33e9b27dac83>:26 train_step  *
        loss        = loss_func(predictions, targets)
    <ipython-input-162-610a34959560>:2 loss_func  *
        loss_energy = tf.reduce_mean((tf.subtract(y_reco[:,0], y_true[:,0]))**2)
    C:\Users\chris\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\util\dispatch.py:201 wrapper
        return target(*args, **kwargs)
    C:\Users\chris\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\ops\array_ops.py:1036 _slice_helper
        return strided_slice(
    C:\Users\chris\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\util\dispatch.py:201 wrapper
        return target(*args, **kwargs)
    C:\Users\chris\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\ops\array_ops.py:1209 strided_slice
        op = gen_array_ops.strided_slice(
    C:\Users\chris\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\ops\gen_array_ops.py:10473 strided_sl

In [ ]:
i

In [ ]:
help(i)